<a href="https://colab.research.google.com/github/skolachi/rongorongo/blob/master/rongorongo_scriptanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://kohaumotu.org/rongorongo_org/concord/concord1.zip
!wget http://kohaumotu.org/rongorongo_org/concord/concord2.zip

--2022-04-03 20:16:18--  http://kohaumotu.org/rongorongo_org/concord/concord1.zip
Resolving kohaumotu.org (kohaumotu.org)... 162.215.248.227
Connecting to kohaumotu.org (kohaumotu.org)|162.215.248.227|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 250143 (244K) [application/zip]
Saving to: ‘concord1.zip’

concord1.zip        100%[===================>] 244.28K  --.-KB/s    in 0.09s   

2022-04-03 20:16:20 (2.74 MB/s) - ‘concord1.zip’ saved [250143/250143]

--2022-04-03 20:16:20--  http://kohaumotu.org/rongorongo_org/concord/concord2.zip
Resolving kohaumotu.org (kohaumotu.org)... 162.215.248.227
Connecting to kohaumotu.org (kohaumotu.org)|162.215.248.227|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229017 (224K) [application/zip]
Saving to: ‘concord2.zip’

concord2.zip        100%[===================>] 223.65K  --.-KB/s    in 0.09s   

2022-04-03 20:16:20 (2.51 MB/s) - ‘concord2.zip’ saved [229017/229017]



In [ ]:
!mkdir rongorongo
!unzip concord1.zip -d rongorongo/ 
!unzip concord2.zip -d rongorongo/
!rm concord1.zip concord2.zip
!cat rongorongo/*.CCD >> rongorongo/fullconcordance.CCD
!wc -l rongorongo/fullconcordance.CCD

Archive:  concord1.zip
  exploding: rongorongo/001.CCD      
  exploding: rongorongo/005.CCD      
  exploding: rongorongo/010.CCD      
  exploding: rongorongo/025.CCD      
  exploding: rongorongo/050.CCD      
  exploding: rongorongo/070.CCD      
Archive:  concord2.zip
  exploding: rongorongo/080.CCD      
  exploding: rongorongo/100.CCD      
  exploding: rongorongo/200.CCD      
  exploding: rongorongo/300.CCD      
  exploding: rongorongo/400.CCD      
  exploding: rongorongo/500.CCD      
  exploding: rongorongo/600.CCD      
  exploding: rongorongo/700.CCD      
16568 rongorongo/fullconcordance.CCD


In [ ]:
import re
def read_corpus(corpusfile):
  return re.findall(r'[A-Z]{1}[a-z]{1}[0-9]{2}\.[0-9]{3}\:([^\n]*)',open(corpusfile).read())

In [ ]:
corpus = read_corpus('rongorongo/fullconcordance.CCD')
len(corpus)
corpus[:10]

['200j.52x-200-8!-200.|1!*(2-3)-48!-1!.10x!-4!.64a-202-280-191b-5-',
 '1.10-5j.52-22by.243-|1t-(1-3)*0!-5j.6-7-93-50b.74x-8-400!-281-1.',
 ' 28x.95j-73af-1-380.|1-(5-8)*(3-5)-421d-95d-87.62-200.700.61-4.',
 ' 100.62-16.62x-202s-|1-0!-0!.10-59f-35.59f.55b-35-460-10.55b-7-',
 '700?-755-90-200-200.|1?-0!*0!-93-739-670-0!-70?-700x?-28-85?-1?-',
 ' 510.7-10.1.11-1-99-|1-0!*0!-200?.200.10f.55b-73f-49f-69-200-6-',
 ' 3-37?-5-306?.37?-5-|1-0!*0!-206?-7-306.3-28.95-642-43:1?-65V-',
 '280?-1-280?-1?-280?-|1?-0!-1.6-522-1.62-600-62.1-522-6.1-380.1.',
 '6.20.10-50-71.76-11-|1V-0!-5-6.74f-596.76-726.77?-400V-70.76-20.',
 '9-8!-1a?-114a.9!-0!-|1!-0!-8!*']

In [30]:
#punctuation = ['-','.',':','\'','*']
samp = ''
def line2sign(line):
  signs = []
  line = re.sub('\([0-9]{1}\-[0-9]{1}\)','-000-',line)
  for c in re.split(r'[\-\.\:\'\*]',line):
    if c != '':
      sign = re.sub('[^0-9]','',c)
      signs.append('0'*(3-len(sign))+sign)
  
  return signs

In [35]:
with open('sign_sequences.txt','w') as f:
  for text in corpus:
    f.write('{}\n'.format(' '.join(line2sign(text))))

In [33]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 6.5 MB 39.1 MB/s 
     |████████████████████████████████| 895 kB 39.8 MB/s 
     |████████████████████████████████| 596 kB 62.4 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [36]:
import os
from pathlib import Path
from tokenizers import BertWordPieceTokenizer
from tokenizers import Tokenizer
from transformers import BertTokenizerFast
from tokenizers.models import Unigram, WordLevel, WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import decoders

In [40]:
def train_tokenizer():
  dictfiles = ['sign_sequences.txt']

  tokenizer = BertWordPieceTokenizer()
  tokenizer.pre_tokenizer = Whitespace()
  tokenizer.decoder = decoders.WordPiece()

  tokenizer.train(files=dictfiles,min_frequency=2,special_tokens=["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"])
  if not os.path.isdir("rongorongoLM"):
    os.mkdir("rongorongoLM")
  tokenizer.save_model("rongorongoLM")

In [41]:
train_tokenizer()
tokenizer = BertTokenizerFast.from_pretrained("rongorongoLM")

In [43]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
  tokenizer=tokenizer,
  file_path="./sign_sequences.txt",
  block_size=128,
)


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


In [44]:
from transformers import BertConfig

config = BertConfig(
    max_position_embeddings=514,
    num_attention_heads=6,
    num_hidden_layers=3,
    type_vocab_size=1,
)

In [45]:
from transformers import BertForMaskedLM

model = BertForMaskedLM(config=config)

print("Number of model parameters: ",model.num_parameters())

Number of model parameters:  45724218


In [46]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.5
)

In [47]:
from transformers import Trainer, TrainingArguments

def train_lm():
    training_args = TrainingArguments(
        output_dir="./rongorongoLM",
        overwrite_output_dir=True,
        num_train_epochs=1,
        per_gpu_train_batch_size=128,
        save_steps=10_000,
        save_total_limit=2,
        prediction_loss_only=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()
    trainer.save_model("./rongorongoLM")

In [48]:
train_lm()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14653
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 115
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is pr

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./rongorongoLM
Configuration saved in ./rongorongoLM/config.json
Model weights saved in ./rongorongoLM/pytorch_model.bin


In [49]:
from transformers import pipeline

fill_mask = pipeline(
  "fill-mask",
  model="./rongorongoLM",
  tokenizer="./rongorongoLM"
)

loading configuration file ./rongorongoLM/config.json
Model config BertConfig {
  "_name_or_path": "./rongorongoLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "bert",
  "num_attention_heads": 6,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file ./rongorongoLM/config.json
Model config BertConfig {
  "_name_or_path": "./rongorongoLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden

In [52]:
corpus[10], ' '.join(line2sign(corpus[10]))

('4-0!-0!-4-99?-0!-0!-|1?-0!-14?-8-600?-1?*385-1*0!-59f-22a-22a*8-',
 '004 000 000 004 099 000 000 001 000 014 008 600 001 385 001 000 059 022 022 008')

In [55]:
fill_mask("004 000 000 004 [MASK] 000 000 001 000 [MASK] 008 600 001 385 001 000 059 022 022 008")

[[{'score': 0.07114849984645844,
   'sequence': '[CLS] 004 000 000 004 001 000 000 001 000 [MASK] 008 600 001 385 001 000 059 022 022 008 [SEP]',
   'token': 28,
   'token_str': '001'},
  {'score': 0.020147668197751045,
   'sequence': '[CLS] 004 000 000 004 004 000 000 001 000 [MASK] 008 600 001 385 001 000 059 022 022 008 [SEP]',
   'token': 34,
   'token_str': '004'},
  {'score': 0.0186594408005476,
   'sequence': '[CLS] 004 000 000 004 003 000 000 001 000 [MASK] 008 600 001 385 001 000 059 022 022 008 [SEP]',
   'token': 37,
   'token_str': '003'},
  {'score': 0.01518901064991951,
   'sequence': '[CLS] 004 000 000 004 076 000 000 001 000 [MASK] 008 600 001 385 001 000 059 022 022 008 [SEP]',
   'token': 31,
   'token_str': '076'},
  {'score': 0.013096786104142666,
   'sequence': '[CLS] 004 000 000 004 000 000 000 001 000 [MASK] 008 600 001 385 001 000 059 022 022 008 [SEP]',
   'token': 39,
   'token_str': '000'}],
 [{'score': 0.06870299577713013,
   'sequence': '[CLS] 004 000 000 0